In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# unzip the dataset
!unzip /content/drive/MyDrive/detectron2_object_detection/RustCOCO.zip -d /content

In [ ]:
pwd

'/content'

In [ ]:
# to be used once model is trained supposed to run at the end
!zip -r '/content/drive/MyDrive/Colab Notebooks/object detection/model' /content/output_directory

In [ ]:
import json
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

def load_dataset_from_json(json_file):
    """
    Load dataset annotations from a JSON file in COCO format.

    Args:
        json_file (str): Path to the JSON file containing the annotations.

    Returns:
        list[dict]: List of dictionary objects where each dictionary corresponds to an image and its annotations.
    """
    with open(json_file, "r") as f:
        data = json.load(f)

    dataset_dicts = []
    for img in data["images"]:
        record = {}
        filename = img["file_name"]
        record["file_name"] = filename  # Image file path
        record["image_id"] = img["id"]  # Image ID
        record["height"] = img["height"]  # Image height
        record["width"] = img["width"]  # Image width

        objs = []
        for ann in data["annotations"]:
            if ann["image_id"] == img["id"]:
                obj = {
                    "bbox": ann["bbox"],  # Bounding box coordinates
                    "bbox_mode": BoxMode.XYWH_ABS,  # Bounding box mode (XYWH_ABS means [x, y, width, height])
                    "category_id": ann["category_id"],  # Category ID of the object
                }
                objs.append(obj)
        record["annotations"] = objs  # Annotations for the current image
        dataset_dicts.append(record)
    return dataset_dicts

# Paths to the training and validation JSON annotation files
train_json_path = '/content/RustCOCO/train_rust/annotations/instances_default.json'
valid_json_path = '/content/RustCOCO/valid_rust/annotations/instances_default.json'

# Register the dataset using COCO format
# This function adds the dataset to the DatasetCatalog and MetadataCatalog
register_coco_instances("my_dataset_train", {}, train_json_path, "/content/RustCOCO/train_rust/images")
register_coco_instances("my_dataset_val", {}, valid_json_path, "/content/RustCOCO/valid_rust/images")

# Set metadata for the datasets, including class names
MetadataCatalog.get("my_dataset_train").set(thing_classes=["Rust", "No Rust"])
MetadataCatalog.get("my_dataset_val").set(thing_classes=["Rust", "No Rust"])

# Get the metadata
metadata = MetadataCatalog.get("my_dataset_train")
metadata_val = MetadataCatalog.get("my_dataset_val")

# Print metadata to verify
print(metadata)
print(metadata_val)

# Now you can use "my_dataset_train" and "my_dataset_val" for training or evaluation


Metadata(name='my_dataset_train', json_file='/content/RustCOCO/train_rust/annotations/instances_default.json', image_root='/content/RustCOCO/train_rust/images', evaluator_type='coco', thing_classes=['Rust', 'No Rust'])
Metadata(name='my_dataset_val', json_file='/content/RustCOCO/valid_rust/annotations/instances_default.json', image_root='/content/RustCOCO/valid_rust/images', evaluator_type='coco', thing_classes=['Rust', 'No Rust'])


In [ ]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow  # Assuming you are using Google Colab

# Load the dataset dictionaries for the validation dataset
dataset_dicts = load_dataset_from_json(valid_json_path)

# Randomly sample 3 images from the dataset for visualization
for d in random.sample(dataset_dicts, 3):
    print(d['file_name'])  # Print the file name of the image being processed
    img = cv2.imread('/content/RustCOCO/valid_rust/images/' + d["file_name"])  # Read the image file
    # Create a Visualizer object to visualize the image and annotations
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata_val, scale=0.5)  # img[:, :, ::-1] converts BGR to RGB
    print(visualizer, d)  # Print the visualizer object and the dictionary for debugging purposes
    vis = visualizer.draw_dataset_dict(d)  # Draw the annotations on the image
    cv2_imshow(vis.get_image()[:, :, ::-1])  # Display the image with annotations (converting RGB back to BGR for display)
``


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [ ]:
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Load the custom dataset
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {},train_json_path, '/content/dataset_1/COCO_Dataset/train/images')

# Configure the model parameters
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2  # Increase this number based on your GPU memory
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = []  # No LR steps, you can add steps if you want LR decay
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Change this to the number of classes in your dataset
cfg.OUTPUT_DIR = "./output_directory"  # Change this to the desired output directory

# Create output directory if it doesn't exist
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Create and train the model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[05/07 11:24:06 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

model_final_f97cb7.pkl: 136MB [00:00, 272MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[05/07 11:24:06 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/07 11:24:18 d2.utils.events]:  eta: 0:02:38  iter: 19  total_loss: 1.762  loss_cls: 0.708  loss_box_reg: 0.6754  loss_rpn_cls: 0.2387  loss_rpn_loc: 0.05045    time: 0.3408  last_time: 0.3160  data_time: 0.0212  last_data_time: 0.0012   lr: 9.7405e-06  max_mem: 1449M
[05/07 11:24:29 d2.utils.events]:  eta: 0:02:39  iter: 39  total_loss: 1.955  loss_cls: 0.6494  loss_box_reg: 0.8956  loss_rpn_cls: 0.2577  loss_rpn_loc: 0.1216    time: 0.3640  last_time: 0.3451  data_time: 0.0153  last_data_time: 0.0148   lr: 1.9731e-05  max_mem: 1511M
[05/07 11:24:37 d2.utils.events]:  eta: 0:02:33  iter: 59  total_loss: 1.559  loss_cls: 0.5606  loss_box_reg: 0.6841  loss_rpn_cls: 0.1721  loss_rpn_loc: 0.06343    time: 0.3753  last_time: 0.3559  data_time: 0.0123  last_data_time: 0.0132   lr: 2.972e-05  max_mem: 1511M
[05/07 11:24:44 d2.utils.events]:  eta: 0:02:25  iter: 79  total_loss: 1.619  loss_cls: 0.5306  loss_box_reg: 0.8093  loss_rpn_cls: 0.1473  loss_rpn_loc: 0.05817    time: 0.3648  last_

Model is trained now we are testing our trained model

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[05/07 11:28:39 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output_directory/model_final.pth ...


In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
import cv2
from google.colab.patches import cv2_imshow  # Assuming you are using Google Colab

# Load the dataset dictionaries for the validation dataset
dataset_dicts = load_dataset_from_json(valid_json_path)

# Randomly sample 3 images from the dataset for visualization
for d in random.sample(dataset_dicts, 3):
    # Print the dictionary (commented out, can be useful for debugging)
    # print(d)

    # Read the image file
    im = cv2.imread('/content/dataset_1/COCO_Dataset/valid/images/' + d["file_name"])

    # Print the image (commented out, can be useful for debugging)
    # print(im)

    # Perform inference on the image using the predictor
    outputs = predictor(im)  # The format of outputs is documented in Detectron2 documentation

    # Create a Visualizer object to visualize the image and predictions
    v = Visualizer(im[:, :, ::-1],  # Convert BGR to RGB
                   metadata=metadata_val,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW  # Grayscale unsegmented areas (for segmentation models)
    )

    # Draw the instance predictions on the image
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Display the image with predictions (converting RGB back to BGR for display)
    cv2_imshow(out.get_image()[:, :, ::-1])
